# Saving and loading populations

Another feature Brush implements is the ability to save and load entire populations.
We use JSON notation to store the population into a file that is human readable. The same way, we can feed an estimator a previous population file to serve as starting point for the evolution.

In this notebook, we will walk through how to use the `save_population` and `load_population` parameters. 

We start by getting a sample dataset and splitting it into `X` and `y`:

In [1]:
import pandas as pd
from pybrush import BrushRegressor

# load data
df = pd.read_csv('../examples/datasets/d_enc.csv')
X = df.drop(columns='label')
y = df['label']

To save the population after finishing the evolution, you nee to set `save_population` parameter to a value different than an empty string. Then, the final population is going to be stored in that specific file.

In this example, we create a temporary file.

In [2]:
import pickle
import os, tempfile

pop_file = os.path.join(tempfile.mkdtemp(), 'population.json')

# set verbosity==2 to see the full report
est = BrushRegressor(
    functions=['SplitBest','Add','Mul','Sin','Cos','Exp','Logabs'],
    max_gens=10,
    objectives=["error", "complexity"],
    save_population=pop_file,
    verbosity=2
)

est.fit(X,y)
y_pred = est.predict(X)
print('score:', est.score(X,y))

Generation 1/10 [//////                                            ]
Train Loss (Med): 10.84173 (54.01542)
Val Loss (Med): 10.84173 (54.01542)
Median Size (Max): 3 (20)
Median complexity (Max): 19 (2163)
Time (s): 0.09850

Generation 2/10 [///////////                                       ]
Train Loss (Med): 10.84173 (38.19256)
Val Loss (Med): 10.84173 (38.19256)
Median Size (Max): 3 (20)
Median complexity (Max): 9 (2163)
Time (s): 0.26875

Generation 3/10 [////////////////                                  ]
Train Loss (Med): 10.43982 (34.84354)
Val Loss (Med): 10.43982 (34.84354)
Median Size (Max): 3 (20)
Median complexity (Max): 9 (2163)
Time (s): 0.46519

Generation 4/10 [/////////////////////                             ]
Train Loss (Med): 10.26326 (17.94969)
Val Loss (Med): 10.26326 (17.94969)
Median Size (Max): 3 (19)
Median complexity (Max): 9 (723)
Time (s): 0.58006

Generation 5/10 [//////////////////////////                        ]
Train Loss (Med): 10.26326 (17.94969)
Val L

Loading a previous population is done providing `load_population` a string value corresponding to a JSON file generated by Brush. In our case, we will use the same file from the previous code block.

After loading the population, we run the evolution for 10 more generations, and we can see that the first generation started from the previous population. This means that the population was successfully saved and loaded.

In [3]:
est = BrushRegressor(
    functions=['SplitBest','Add','Mul','Sin','Cos','Exp','Logabs'],
    load_population=pop_file,
    max_gens=10,
    verbosity=1
)

est.fit(X,y)
y_pred = est.predict(X)
print('score:', est.score(X,y))

Loaded population from /tmp/tmpvo0quw4a/population.json of size = 200
Completed 100% [====================]
saving final population as archive...
score: 0.8900709681238513


## Saving just the archive

In case you want to use another expression rather than the final `best_estimator_`, brush provides the archive option.

The archive is just the pareto front from the population. You can use `predict_archive` (and `predict_proba_archive` if using a `BrushClassifier`) to call the prediction methods for the entire archive, instead of the selected best individual.

But first, you need to enable this option with `use_arch=True`. When set to False, it will store the entire final population

In [4]:
est = BrushRegressor(
    functions=['SplitBest','Add','Mul','Sin','Cos','Exp','Logabs'],
    load_population=pop_file,
    use_arch=True,
    max_gens=10,
    verbosity=1
)

est.fit(X,y)

# accessing first expression from the archive. It is serialized as a dict
print(est.archive_[0]['fitness'])

Loaded population from /tmp/tmpvo0quw4a/population.json of size = 200
Completed 100% [====================]
{'complexity': 240, 'crowding_dist': 3.4028234663852886e+38, 'dcounter': 0, 'depth': 3, 'dominated': [], 'linear_complexity': 24, 'loss': 9.675766944885254, 'loss_v': 9.675766944885254, 'rank': 1, 'size': 17, 'values': [9.675766944885254, 17.0], 'weights': [-1.0, -1.0], 'wvalues': [-9.675766944885254, -17.0]}


You can open the serialized file and change individuals' programs manually.

This also allow us to have checkpoints in the execution.

## Using population files with classification

To give another example, we do a two-step fit in the cells below.

First, we run the evolution and save the population to a file; then, we load it and keep evolving the individuals.

What is different though is that the first run is optimizing `error` and `complexity`, and the second run is optimizing `average_precision_score` and `linear_complexity`.

In [5]:
from pybrush import BrushClassifier

# load data
df = pd.read_csv('../examples/datasets/d_analcatdata_aids.csv')
X = df.drop(columns='target')
y = df['target']

pop_file = os.path.join(tempfile.mkdtemp(), 'population.json')

est = BrushClassifier(
    functions=['SplitBest','Add','Mul','Sin','Cos','Exp','Logabs'],
    max_gens=10,
    objectives=["error", "complexity"],
    scorer="log",
    save_population=pop_file,
    verbosity=2
)

est.fit(X,y)
print(est.best_estimator_.get_model())

y_pred = est.predict(X)
print('score:', est.score(X,y))

Generation 1/10 [//////                                            ]
Train Loss (Med): 0.54847 (0.69315)
Val Loss (Med): 0.54847 (0.69315)
Median Size (Max): 5 (10)
Median complexity (Max): 6 (270)
Time (s): 0.04547

Generation 2/10 [///////////                                       ]
Train Loss (Med): 0.54846 (0.69315)
Val Loss (Med): 0.54846 (0.69315)
Median Size (Max): 5 (10)
Median complexity (Max): 6 (90)
Time (s): 0.07358

Generation 3/10 [////////////////                                  ]
Train Loss (Med): 0.54846 (0.69315)
Val Loss (Med): 0.54846 (0.69315)
Median Size (Max): 5 (10)
Median complexity (Max): 6 (279)
Time (s): 0.10416

Generation 4/10 [/////////////////////                             ]
Train Loss (Med): 0.54846 (0.69315)
Val Loss (Med): 0.54846 (0.69315)
Median Size (Max): 2 (10)
Median complexity (Max): 2 (279)
Time (s): 0.13638

Generation 5/10 [//////////////////////////                        ]
Train Loss (Med): 0.54846 (0.69315)
Val Loss (Med): 0.54846 (0.6

In [6]:
est = BrushClassifier(
    functions=['SplitBest','Add','Mul','Sin','Cos','Exp','Logabs'],
    #load_population=pop_file,
    objectives=["error", "linear_complexity"],
    scorer="average_precision_score",
    max_gens=10,
    verbosity=2
)

est.fit(X,y)
print(est.best_estimator_.get_model())

y_pred = est.predict(X)
print('score:', est.score(X,y))

Generation 1/10 [//////                                            ]
Train Loss (Med): 0.46115 (0.31675)
Val Loss (Med): 0.46115 (0.31675)
Median Size (Max): 5 (9)
Median complexity (Max): 6 (120)
Time (s): 0.07469

Generation 2/10 [///////////                                       ]
Train Loss (Med): 0.55038 (0.31675)
Val Loss (Med): 0.55038 (0.31675)
Median Size (Max): 5 (9)
Median complexity (Max): 6 (648)
Time (s): 0.12616

Generation 3/10 [////////////////                                  ]
Train Loss (Med): 0.45385 (0.00000)
Val Loss (Med): 0.45385 (0.00000)
Median Size (Max): 1 (8)
Median complexity (Max): 1 (648)
Time (s): 0.16134

Generation 4/10 [/////////////////////                             ]
Train Loss (Med): 0.75212 (0.00000)
Val Loss (Med): 0.75212 (0.00000)
Median Size (Max): 1 (9)
Median complexity (Max): 1 (30)
Time (s): 0.20292

Generation 5/10 [//////////////////////////                        ]
Train Loss (Med): 0.75212 (0.00000)
Val Loss (Med): 0.75212 (0.00000